In [ ]:

import pandas as pd
import nltk
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.corpus import stopwords
import csv
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import gensim
from gensim.models import Word2Ve

In [2]:
# Gerekli NLTK kaynaklarını indirilmesi 
nltk.download('punkt', quiet=True)
nltk.download('stopwords', quiet=True)

True

In [3]:
import pandas as pd

# Başlık satırını otomatik olarak kullan
df = pd.read_csv('pg_catalog.csv', encoding='utf-8')

# Sütun adlarını göster (emin olmak için)
print("Sütunlar:", df.columns.tolist())

# 'Title' sütununu listeye çevir
texts = df['Title'].dropna().astype(str).tolist()

# İlk metnin ilk 500 karakterini yazdır
if texts:
    print("\nVerinin ilk 500 karakteri:")
    print(texts[0][:500])
else:
    print("Veri listesi boş.")


Sütunlar: ['Text#', 'Type', 'Issued', 'Title', 'Language', 'Authors', 'Subjects', 'LoCC', 'Bookshelves']

Verinin ilk 500 karakteri:
The Declaration of Independence of the United States of America


In [4]:
# [3] Cümlelere ayırma ve ilk 10 cümleyi gösterme
sentences = []
for text in texts:
    if isinstance(text, str):
        sentences.extend(sent_tokenize(text))
    else:
        continue
print("\nİlk 10 cümle:")
print(sentences[:10])


İlk 10 cümle:
['The Declaration of Independence of the United States of America', 'The United States Bill of Rights\r\nThe Ten Original Amendments to the Constitution of the United States', "John F. Kennedy's Inaugural Address", "Lincoln's Gettysburg Address\r\nGiven November 19, 1863 on the battlefield near Gettysburg, Pennsylvania, USA", 'The United States Constitution', 'Give Me Liberty or Give Me Death', 'The Mayflower Compact', "Abraham Lincoln's Second Inaugural Address", "Abraham Lincoln's First Inaugural Address", 'The King James Version of the Bible']


In [5]:
# [4] Türkçe stopwords listesini alma ve ilk 50’sini yazdırma
turkish_stopwords = set([
    've', 'ile', 'de', 'da', 'ki', 'kadar', 'için', 'ama', 'ya', 'veya',
    'bir', 'bu', 'şu', 'o', 'ne', 'nasıl', 'niye', 'hangi', 'her', 'tüm',
    'mi', 'mı', 'mu', 'mü', 'ise', 'değil', 'çok', 'az'
])
stop_words_list = list(turkish_stopwords)
print("\nTürkçe Stopwords (ilk 50):")
print(stop_words_list[:50])


Türkçe Stopwords (ilk 50):
['mı', 'için', 'mu', 'her', 'bu', 'ki', 'mü', 'kadar', 'o', 'de', 'az', 'şu', 'veya', 'ne', 'tüm', 'ya', 'çok', 'mi', 'ise', 'değil', 'ile', 'da', 'hangi', 'niye', 've', 'nasıl', 'ama', 'bir']


In [6]:
# [5] Stemleme fonksiyonunu başlatma
def simple_turkish_stem(token):
    suffixes = ['ler', 'lar', 'in', 'ın', 'un', 'ün', 'de', 'da', 'ki', 'e', 'a']
    for suffix in suffixes:
        if token.endswith(suffix):
            return token[:-len(suffix)]
    return token

In [7]:
# [6] Ön işleme fonksiyonunu tanımlama
def preprocess_sentence(sentence):
    if not isinstance(sentence, str):
        return [], []
    tokens = word_tokenize(sentence)
    filtered_tokens = [token.lower() for token in tokens if token.isalpha() and token.lower() not in turkish_stopwords]
    lemmatized_tokens = filtered_tokens  # Zembereksiz: filtrelenmiş kelimeler
    stemmed_tokens = [simple_turkish_stem(token) for token in filtered_tokens]
    return lemmatized_tokens, stemmed_tokens


In [8]:
# [7-8] Cümleleri işleme (fonksiyonlu)
tokenized_corpus_lemmatized = []
tokenized_corpus_stemmed = []
for sentence in sentences:
    lemmatized_tokens, stemmed_tokens = preprocess_sentence(sentence)
    tokenized_corpus_lemmatized.append(lemmatized_tokens)
    tokenized_corpus_stemmed.append(stemmed_tokens)

In [9]:
# İlk işlenen cümlenin çıktısını göster
print("\nİlk İşlenen Cümle (Fonksiyonlu):")
print(f"Ham: {sentences[0]}")
print(f"Lemmatized: {tokenized_corpus_lemmatized[0]}")
print(f"Stemmed: {tokenized_corpus_stemmed[0]}")


İlk İşlenen Cümle (Fonksiyonlu):
Ham: The Declaration of Independence of the United States of America
Lemmatized: ['the', 'declaration', 'of', 'independence', 'of', 'the', 'united', 'states', 'of', 'america']
Stemmed: ['th', 'declaration', 'of', 'independenc', 'of', 'th', 'united', 'states', 'of', 'americ']


In [10]:
# [9] Lemmatize edilmiş cümleleri CSV’ye kaydetme
with open("lemmatized_sentences.csv", mode="w", newline="", encoding="utf-8") as file:
    writer = csv.writer(file)
    for tokens in tokenized_corpus_lemmatized:
        writer.writerow([' '.join(tokens)])

In [11]:
# [10] Stemlenmiş cümleleri CSV’ye kaydetme
with open("stemmed_sentences.csv", mode="w", newline="", encoding="utf-8") as file:
    writer = csv.writer(file)
    for tokens in tokenized_corpus_stemmed:
        writer.writerow([' '.join(tokens)])

In [12]:
# [11] İlk 5 cümlenin ham, lemmatize ve stemlenmiş hallerini yazdırma
print("\nİlk 5 Cümle Karşılaştırması (Fonksiyonlu):")
for i in range(min(5, len(sentences))):
    print(f"Cümle {i+1} - Ham: {sentences[i]}")
    print(f"Cümle {i+1} - Lemmatized: {tokenized_corpus_lemmatized[i]}")
    print(f"Cümle {i+1} - Stemmed: {tokenized_corpus_stemmed[i]}")
    print()


İlk 5 Cümle Karşılaştırması (Fonksiyonlu):
Cümle 1 - Ham: The Declaration of Independence of the United States of America
Cümle 1 - Lemmatized: ['the', 'declaration', 'of', 'independence', 'of', 'the', 'united', 'states', 'of', 'america']
Cümle 1 - Stemmed: ['th', 'declaration', 'of', 'independenc', 'of', 'th', 'united', 'states', 'of', 'americ']

Cümle 2 - Ham: The United States Bill of Rights
The Ten Original Amendments to the Constitution of the United States
Cümle 2 - Lemmatized: ['the', 'united', 'states', 'bill', 'of', 'rights', 'the', 'ten', 'original', 'amendments', 'to', 'the', 'constitution', 'of', 'the', 'united', 'states']
Cümle 2 - Stemmed: ['th', 'united', 'states', 'bill', 'of', 'rights', 'th', 'ten', 'original', 'amendments', 'to', 'th', 'constitution', 'of', 'th', 'united', 'states']

Cümle 3 - Ham: John F. Kennedy's Inaugural Address
Cümle 3 - Lemmatized: ['john', 'kennedy', 'inaugural', 'address']
Cümle 3 - Stemmed: ['john', 'kennedy', 'inaugural', 'address']

Cümle

In [13]:
# [16-17] Ayrıntılı for döngüsü ile kelimeleri tokenleştirme ve filtreleme
filtered_sentences = []
for sentence in sentences:
    tokens = word_tokenize(sentence)
    filtered_tokens = []
    for token in tokens:
        if token.isalpha():
            token_lower = token.lower()
            if token_lower not in turkish_stopwords:
                filtered_tokens.append(token_lower)
    filtered_sentences.append(filtered_tokens)

In [14]:
# [18] Lemmatize edilmiş cümleleri oluşturma 
tokenized_corpus_lemmatized_detailed = filtered_sentences

In [15]:
# [19] Ayrıntılı stemleme
tokenized_corpus_stemmed_detailed = []
for filtered_tokens in filtered_sentences:
    stemmed_tokens = [simple_turkish_stem(token) for token in filtered_tokens]
    tokenized_corpus_stemmed_detailed.append(stemmed_tokens)
print("\nAyrıntılı Stemlenmiş Cümleler (ilk 10):")
print(tokenized_corpus_stemmed_detailed[:10])


Ayrıntılı Stemlenmiş Cümleler (ilk 10):
[['th', 'declaration', 'of', 'independenc', 'of', 'th', 'united', 'states', 'of', 'americ'], ['th', 'united', 'states', 'bill', 'of', 'rights', 'th', 'ten', 'original', 'amendments', 'to', 'th', 'constitution', 'of', 'th', 'united', 'states'], ['john', 'kennedy', 'inaugural', 'address'], ['lincoln', 'gettysburg', 'address', 'given', 'november', 'on', 'th', 'battlefield', 'near', 'gettysburg', 'pennsylvani', 'us'], ['th', 'united', 'states', 'constitution'], ['giv', 'm', 'liberty', 'or', 'giv', 'm', 'death'], ['th', 'mayflower', 'compact'], ['abraham', 'lincoln', 'second', 'inaugural', 'address'], ['abraham', 'lincoln', 'first', 'inaugural', 'address'], ['th', 'king', 'james', 'version', 'of', 'th', 'bibl']]


In [16]:
# [20] Ayrıntılı stemlenmiş cümleleri CSV’ye kaydetme
with open("stemmed_sentences_detailed.csv", mode="w", newline="", encoding="utf-8") as file:
    writer = csv.writer(file)
    for tokens in tokenized_corpus_stemmed_detailed:
        writer.writerow([' '.join(tokens)])

In [17]:
# [21] İlk 5 cümlenin ayrıntılı işlenmiş hallerini yazdırma
print("\nİlk 5 Cümle Karşılaştırması (Ayrıntılı):")
for i in range(min(5, len(sentences))):
    print(f"Cümle {i+1} - Ham: {sentences[i]}")
    print(f"Cümle {i+1} - Lemmatized: {tokenized_corpus_lemmatized_detailed[i]}")
    print(f"Cümle {i+1} - Stemmed: {tokenized_corpus_stemmed_detailed[i]}")
    print()



İlk 5 Cümle Karşılaştırması (Ayrıntılı):
Cümle 1 - Ham: The Declaration of Independence of the United States of America
Cümle 1 - Lemmatized: ['the', 'declaration', 'of', 'independence', 'of', 'the', 'united', 'states', 'of', 'america']
Cümle 1 - Stemmed: ['th', 'declaration', 'of', 'independenc', 'of', 'th', 'united', 'states', 'of', 'americ']

Cümle 2 - Ham: The United States Bill of Rights
The Ten Original Amendments to the Constitution of the United States
Cümle 2 - Lemmatized: ['the', 'united', 'states', 'bill', 'of', 'rights', 'the', 'ten', 'original', 'amendments', 'to', 'the', 'constitution', 'of', 'the', 'united', 'states']
Cümle 2 - Stemmed: ['th', 'united', 'states', 'bill', 'of', 'rights', 'th', 'ten', 'original', 'amendments', 'to', 'th', 'constitution', 'of', 'th', 'united', 'states']

Cümle 3 - Ham: John F. Kennedy's Inaugural Address
Cümle 3 - Lemmatized: ['john', 'kennedy', 'inaugural', 'address']
Cümle 3 - Stemmed: ['john', 'kennedy', 'inaugural', 'address']

Cümle 4

In [18]:

# [2-3] Lemmatize metinleri oluşturma ve ilk 3’ünü gösterme
lemmatized_texts = [' '.join(tokens) for tokens in tokenized_corpus_lemmatized]
print("\nİlk 3 lemmatized metin:")
print(lemmatized_texts[:3])


İlk 3 lemmatized metin:
['the declaration of independence of the united states of america', 'the united states bill of rights the ten original amendments to the constitution of the united states', 'john kennedy inaugural address']


In [19]:
# [4] TF-IDF vektörizasyon
vectorizer = TfidfVectorizer()
tfidf_matrix = vectorizer.fit_transform(lemmatized_texts)
feature_names = vectorizer.get_feature_names_out()
tfidf_df = pd.DataFrame(tfidf_matrix.toarray(), columns=feature_names)

In [20]:
# [5] İlk 5 cümlenin TF-IDF skorlarını yazdırma
print("\nİlk 5 cümlenin TF-IDF skorları:")
print(tfidf_df.head())


İlk 5 cümlenin TF-IDF skorları:
   aab  aabenserraagin  aada  aade  aadolf  aalberg  aalloilta  aalloissa  \
0  0.0             0.0   0.0   0.0     0.0      0.0        0.0        0.0   
1  0.0             0.0   0.0   0.0     0.0      0.0        0.0        0.0   
2  0.0             0.0   0.0   0.0     0.0      0.0        0.0        0.0   
3  0.0             0.0   0.0   0.0     0.0      0.0        0.0        0.0   
4  0.0             0.0   0.0   0.0     0.0      0.0        0.0        0.0   

   aallonhalkoja  aallot  ...  高士傳  鬼谷四友志  鬼谷子  魏鄭公諫錄  鶯鶯傳  鹽鐵論  麟兒報  黃繡球  \
0            0.0     0.0  ...  0.0    0.0  0.0    0.0  0.0  0.0  0.0  0.0   
1            0.0     0.0  ...  0.0    0.0  0.0    0.0  0.0  0.0  0.0  0.0   
2            0.0     0.0  ...  0.0    0.0  0.0    0.0  0.0  0.0  0.0  0.0   
3            0.0     0.0  ...  0.0    0.0  0.0    0.0  0.0  0.0  0.0  0.0   
4            0.0     0.0  ...  0.0    0.0  0.0    0.0  0.0  0.0  0.0  0.0   

   黄帝宅經  龍川詞  
0   0.0  0.0  
1   0.0  0.

In [21]:
# İlk cümle için TF-IDF skorlarını al
first_sentence_vector = tfidf_df.iloc[0]
top_5_words = first_sentence_vector.sort_values(ascending=False).head(5)
print("\nİlk cümlede en yüksek TF-IDF skoruna sahip 5 kelime:")
print(top_5_words)


İlk cümlede en yüksek TF-IDF skoruna sahip 5 kelime:
declaration     0.512973
independence    0.467458
of              0.350719
states          0.346484
united          0.345850
Name: 0, dtype: float64


In [22]:
# [6] "sorun" kelimesi için cosine benzerlik analizi
try:
    sorun_index = list(feature_names).index('sorun')
    sorun_vector = tfidf_matrix[:, sorun_index].toarray()
    tfidf_vectors = tfidf_matrix.toarray()
    similarities = cosine_similarity(sorun_vector.T, tfidf_vectors.T)
    similarities = similarities.flatten()
    top_5_indices = similarities.argsort()[-6:][:-1]
    print("\n'sorun' kelimesine en benzer 5 kelime:")
    for index in top_5_indices:
        print(f"{feature_names[index]}: {similarities[index]:.4f}")
except ValueError:
    print("\n'sorun' kelimesi veri setinde bulunamadı.")


'sorun' kelimesi veri setinde bulunamadı.


In [23]:

# Word2Vec modeli eğitmek için parametreler
parameters = [
    {'model_type': 'cbow', 'window': 2, 'vector_size': 100},
    {'model_type': 'skipgram', 'window': 2, 'vector_size': 100},
    {'model_type': 'cbow', 'window': 4, 'vector_size': 100},
    {'model_type': 'skipgram', 'window': 4, 'vector_size': 100},
    {'model_type': 'cbow', 'window': 2, 'vector_size': 300},
    {'model_type': 'cbow', 'window': 4, 'vector_size': 300},
    {'model_type': 'skipgram', 'window': 4, 'vector_size': 300}
]


In [24]:
# Fonksiyon ile Word2Vec modeli eğitme ve kaydetme
def train_and_save_model(corpus, params, model_name):
    model = Word2Vec(corpus, vector_size=params['vector_size'],
                     window=params['window'], min_count=1,
                     sg=1 if params['model_type'] == 'skipgram' else 0)
    model.save(f"{model_name}_{params['model_type']}_window{params['window']}_dim{params['vector_size']}.model")
    print(f"{model_name}_{params['model_type']}_window{params['window']}_dim{params['vector_size']}.model saved!")

In [25]:
from gensim.models import Word2Vec

In [26]:
# Lemmatize edilmiş corpus ile modelleri eğitme ve kaydetme
for param in parameters:
    train_and_save_model(tokenized_corpus_lemmatized, param, "lemmatized_model")
    # Stemlenmiş corpus ile modelleri eğitme ve kaydetme
for param in parameters:
    train_and_save_model(tokenized_corpus_stemmed, param, "stemmed_model")

lemmatized_model_cbow_window2_dim100.model saved!
lemmatized_model_skipgram_window2_dim100.model saved!
lemmatized_model_cbow_window4_dim100.model saved!
lemmatized_model_skipgram_window4_dim100.model saved!
lemmatized_model_cbow_window2_dim300.model saved!
lemmatized_model_cbow_window4_dim300.model saved!
lemmatized_model_skipgram_window4_dim300.model saved!
stemmed_model_cbow_window2_dim100.model saved!
stemmed_model_skipgram_window2_dim100.model saved!
stemmed_model_cbow_window4_dim100.model saved!
stemmed_model_skipgram_window4_dim100.model saved!
stemmed_model_cbow_window2_dim300.model saved!
stemmed_model_cbow_window4_dim300.model saved!
stemmed_model_skipgram_window4_dim300.model saved!


In [27]:
# [17] Üç model yükleme ve "sorun" kelimesi için en benzer 3 kelimeyi yazdırma
# Model dosyalarını yükleme
try:
    model_1 = Word2Vec.load("lemmatized_model_cbow_window2_dim100.model")
    model_2 = Word2Vec.load("stemmed_model_skipgram_window2_dim300.model")
    model_3 = Word2Vec.load("lemmatized_model_skipgram_window2_dim300.model")

    # 'sorun' kelimesi ile en benzer 3 kelimeyi ve skorlarını yazdırma
    def print_similar_words(model, model_name):
        try:
            similarity = model.wv.most_similar('sorun', topn=3)
            print(f"\n{model_name} Modeli - 'sorun' ile En Benzer 3 Kelime:")
            for word, score in similarity:
                print(f"Kelime: {word}, Benzerlik Skoru: {score:.4f}")
        except KeyError:
            print(f"\n{model_name} Modeli - 'sorun' kelimesi modelde bulunamadı.")

    # 3 model için benzer kelimeleri yazdırma
    print_similar_words(model_1, "Lemmatized CBOW Window 2 Dim 100")
    print_similar_words(model_2, "Stemmed Skipgram Window 2 Dim 300")
    print_similar_words(model_3, "Lemmatized Skipgram Window 2 Dim 300")
except FileNotFoundError:
    print("\nModellerden biri bulunamadı. Önce modelleri eğitip kaydedin.")


Modellerden biri bulunamadı. Önce modelleri eğitip kaydedin.
